In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
sys.setrecursionlimit(5000)

In [3]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

from compute_ET_green import compute_et_green, calculate_band_std_dev
from compute_ET_blue import (
    compute_et_blue,
    compute_volumetric_et_blue,
    postprocess_et_blue,
)
from add_walter_irrigation_estimates import filter_and_estimate_irrigation

from utils import ee_utils, date_utils
from utils.ee_utils import back_to_float, back_to_int, export_image_to_asset

from typing import List, Tuple

import ee
import geemap

In [4]:
ee.Initialize(project="thurgau-irrigation")

## Define the region of interest


In [5]:
# For the canton of Thurgau
cantonal_borders_asset = (
    "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
)

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

## Importing all assets: land use maps, vegetation periods, WaPOR ET data.


In [6]:
years_to_process = range(2018, 2024)

double_cropping_image_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/vegetation_periods_2018-2023"
)

double_cropping_list = double_cropping_image_collection.toList(
    double_cropping_image_collection.size()
)

WaPOR_assets = (
    "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_monthly"
)

WaPOR_et_collections = ee_utils.merge_collections(
    years=years_to_process, asset_name=WaPOR_assets
)

WaPOR_et_collections = WaPOR_et_collections.map(lambda image: back_to_float(image, 100))

WaPOR_et_collections_list = WaPOR_et_collections.toList(WaPOR_et_collections.size())

## Sanity check, uncomment to verify things
### Filtering out potentially irrigated crops and refeence rainfed fields


In [7]:
# nutzung_collection = ee.FeatureCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2018_area"
# )

# double_cropping_image_2018 = double_cropping_image_collection.first()

# # Add double cropping information to the feature collection
# nutzung_collection_with_double_crop = add_double_cropping_info(
#     nutzung_collection, double_cropping_image_2018
# )

# not_irrigated_crops = get_crops_to_exclude()
# rainfed_crops = get_rainfed_reference_crops()

# exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

# potentially_rainfed, rainfed_fields = filter_crops(
#     nutzung_collection_with_double_crop, exclude_filter, rainfed_filter
# )

# print("Filtered fields count:", potentially_rainfed.size().getInfo())
# print("Rainfed reference fields count:", rainfed_fields.size().getInfo())

### Computing ET green and exporting the results


In [8]:
# et_image = ee.Image(WaPOR_et_collections.toList(WaPOR_et_collections.size()).get(20))
# jurisdiction_polygons = ee.FeatureCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/thurgau_jurisdictions_2018_wgs84"
# )

# et_green_result = compute_et_green(et_image, rainfed_fields, jurisdiction_polygons)

In [9]:
# Map = geemap.Map()

# Map.centerObject(aoi_geometry, 12)
# options = {
#     "bands": ["ET_green"],
#     "min": 0,
#     "max": 10000,
#     "palette": ["blue", "green", "yellow", "red"],
# }
# Map.addLayer(et_green_result, options, "ET_green")
# Map.addLayer(jurisdiction_polygons, {"color": "white"}, "Jurisdictions")
# options_et = {
#     "bands": ["downscaled"],
#     "min": 0,
#     "max": 100,
#     "palette": ["blue", "green", "yellow", "red"],
# }
# Map.addLayer(et_image, options_et, "ET")

# Map

### Computing ET green and exporting to asset


In [10]:
# TODO: refactor the nested for loop

IMAGES_PER_YEAR = 12  # Monthly

not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

tasks = []

total_tasks = 0

for year in years_to_process:
    j = year - 2018
    nutzung_collection_year = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
    )

    jurisdiction_polygons = ee.FeatureCollection(
        f"projects/thurgau-irrigation/assets/Thurgau/thurgau_jurisdictions_{year}_wgs84"
    )
    double_cropping_image_year = ee.Image(double_cropping_list.get(j))

    nutzung_collection_with_double_crop_year = add_double_cropping_info(
        nutzung_collection_year, double_cropping_image_year
    )

    potentially_rainfed_year, rainfed_fields_year = filter_crops(
        nutzung_collection_with_double_crop_year, exclude_filter, rainfed_filter
    )

    year_tasks = 0

    for i in range(IMAGES_PER_YEAR):
        et_image_year = ee.Image(WaPOR_et_collections_list.get(j * IMAGES_PER_YEAR + i))
        et_green_result_year = compute_et_green(
            et_image_year, rainfed_fields_year, jurisdiction_polygons
        )

        # dekadal = i % 3 + 1
        # month = i // 3 + 1
        month = i + 1

        task_name = f"ET_green_jurisdiction_monthly_{year}-{month:02d}"
        asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_monthly_2018-2023/{task_name}"

        task = export_image_to_asset(
            et_green_result_year, asset_id, task_name, year, aoi_buffered
        )
        tasks.append(task)
        year_tasks += 1
        total_tasks += 1

    print(f"Year {year} processing complete. Started {year_tasks} tasks.")

print(f"All processing complete. Total tasks started: {total_tasks}")

Exporting ET_green_jurisdiction_monthly_2018-01 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_monthly_2018-2023/ET_green_jurisdiction_monthly_2018-01
Exporting ET_green_jurisdiction_monthly_2018-02 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_monthly_2018-2023/ET_green_jurisdiction_monthly_2018-02
Exporting ET_green_jurisdiction_monthly_2018-03 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_monthly_2018-2023/ET_green_jurisdiction_monthly_2018-03
Exporting ET_green_jurisdiction_monthly_2018-04 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_monthly_2018-2023/ET_green_jurisdiction_monthly_2018-04
Exporting ET_green_jurisdiction_monthly_2018-05 for 2018 to projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_monthly_2018-2023/ET_green_jurisdiction_monthly_2018-05
Exporting ET_green_jurisdiction_monthly_2018-06 for 2018 to projects/thurgau-irrigation/assets/

## Exporting ET Blue maps


In [ ]:
# et_green_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/ET_green_jurisdiction_2018-2022"
# ).map(lambda image: back_to_float(image, 100))

# et_green_collection_list = et_green_collection.toList(et_green_collection.size())

In [ ]:
# tasks = []

# total_tasks = 0

# previous_ET_image = None

# for year in years_to_process:
#     j = year - 2018

#     year_tasks = 0

#     for i in range(36):
#         et_image_year = ee.Image(WaPOR_et_collections_list.get(j * 36 + i))
#         et_green_year = ee.Image(et_green_collection_list.get(j * 36 + i))

#         et_blue = compute_et_blue(et_image_year, et_green_year)

#         # Multiply by 100 and cast to int to save space
#         et_blue = back_to_int(et_blue, 100)

#         dekadal = i % 3 + 1
#         month = i // 3 + 1

#         task_name = f"ET_raw_{year}-{month:02d}_D{dekadal}"

#         asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_2018-2022/{task_name}"

#         task = export_image_to_asset(et_blue, asset_id, task_name, year, aoi_buffered)
#         tasks.append(task)
#         year_tasks += 1
#         total_tasks += 1

#     print(f"Year {year} processing complete. Started {year_tasks} tasks.")

# print(f"All processing complete. Total tasks started: {total_tasks}")

## Postprocess ET_blue

In [ ]:
# tasks = []

# total_tasks = 0

# et_blue_raw = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_2018-2022"
# ).map(lambda image: back_to_float(image, 100))

# et_blue_raw_list = et_blue_raw.toList(et_blue_raw.size())

# et_blue_previous = None

# for year in years_to_process:
#     j = year - 2018

#     year_tasks = 0

#     for i in range(36):
#         et_green_year = ee.Image(et_green_collection_list.get(j * 36 + i))
#         et_blue_present = ee.Image(et_blue_raw_list.get(j * 36 + i))

#         if not et_blue_previous:
#             et_blue_previous = et_blue_present

#         threshold = calculate_band_std_dev(et_green_year, "ET_green")

#         et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

#         et_blue_m3 = compute_volumetric_et_blue(et_blue)

#         et_blue = et_blue.addBands(et_blue_m3)

#         previous_ET_image = et_blue.select("ET_blue")

#         # Multiply by 100 and cast to int to save space
#         et_blue = back_to_int(et_blue, 100)

#         dekadal = i % 3 + 1
#         month = i // 3 + 1

#         task_name = f"ET_postprocessed_{year}-{month:02d}_D{dekadal}"

#         asset_id = f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_2018-2022/{task_name}"

#         task = export_image_to_asset(et_blue, asset_id, task_name, year, aoi_buffered)
#         tasks.append(task)
#         year_tasks += 1
#         total_tasks += 1

#     print(f"Year {year} processing complete. Started {year_tasks} tasks.")

# print(f"All processing complete. Total tasks started: {total_tasks}")

## Exploring Nutzungsflaechen


In [ ]:
# nutzung_collection = ee.FeatureCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2018_area"
# )

In [ ]:
# def get_unique_nutzung_values(feature_collection: ee.FeatureCollection) -> list:
#     """
#     Get all unique values of the 'nutzung' property from a feature collection.

#     Args:
#         feature_collection (ee.FeatureCollection): The input feature collection.

#     Returns:
#         list: A list of unique 'nutzung' values.
#     """
#     # Get distinct values of 'nutzung'
#     distinct_values = feature_collection.distinct('nutzung')

#     # Aggregate the distinct values into a list
#     unique_values = distinct_values.aggregate_array('nutzung')

#     # Get the result as a Python list
#     return unique_values.getInfo()

# unique_nutzung_values = get_unique_nutzung_values(nutzung_collection)

# print("Unique 'nutzung' values:")
# for value in unique_nutzung_values:
#     print(value)

In [ ]:
# def export_feature_collection(collection: ee.FeatureCollection, task_name: str, asset_id: str):
#     """
#     Export the feature collection to an Earth Engine asset.

#     Args:
#         collection: The feature collection to export.
#         year: The year of the feature collection.
#         task_name: The name of the export task.
#         asset_id: The asset ID to export to.
#     """
#     task = ee.batch.Export.table.toAsset(
#         collection=collection,
#         description=task_name,
#         assetId=asset_id,
#     )
#     task.start()

# for year in years_to_process:
#     nutzung_collection_year = ee.FeatureCollection(
#         f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
#     )

#     # Apply the filter and estimation function
#     validation_nutzung_collection_year = filter_and_estimate_irrigation(nutzung_collection_year)

#     task_name = f"irrigation_estimates_{year}"
#     asset_id = f"projects/thurgau-irrigation/assets/Thurgau/Walter_Koch_irrigation_estimates_2018-2022/{task_name}"

#     export_feature_collection(validation_nutzung_collection_year, task_name, asset_id)

# print("All processing complete.")

In [ ]:
# image_collection = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_2018-2022").map(lambda image: back_to_float(image, 100))

# image_list = image_collection.toList(image_collection.size())

# image = ee.Image(image_list.get(15))

# # image = ee.Image(WaPOR_et_collections_list.get(15)).subtract(ee.Image(et_green_collection_list.get(15)))

# Map = geemap.Map()

# Map.centerObject(aoi_geometry, 12)

# vis_params = {
#     "bands": ["ET_blue"],
#     "min": -1,
#     "max": 2,
#     "palette": ["white", "blue", "green", "yellow", "red"],
# }

# Map.addLayer(image, vis_params, "ET_blue_postprocessed")

# Map

In [ ]:
# et_blue_raw = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/ET_blue_raw_2018-2022"
# ).map(lambda image: back_to_float(image, 100))

# et_blue_raw_list = et_blue_raw.toList(et_blue_raw.size())

In [ ]:
# et_blue_present = ee.Image(et_blue_raw_list.get(20))
# et_blue_previous = ee.Image(et_blue_raw_list.get(19))

# et_green_year = ee.Image(et_green_collection_list.get(20))

# threshold = calculate_band_std_dev(et_green_year, "ET_green")

# et_blue = postprocess_et_blue(et_blue_present, et_blue_previous, threshold)

# et_blue_m3 = compute_volumetric_et_blue(et_blue)

In [ ]:
# image_postprocessed = ee.ImageCollection("projects/thurgau-irrigation/assets/Thurgau/ET_blue_postprocessed_2018-2022")

# image_postprocessed.first().date().format("YYYY-MM-dd").getInfo()